In [ ]:
!pip install skorch
!pip install git+https://github.com/modAL-python/modAL.git

  Cloning https://github.com/modAL-python/modAL.git to /tmp/pip-req-build-vwmjlevx
  Running command git clone --filter=blob:none --quiet https://github.com/modAL-python/modAL.git /tmp/pip-req-build-vwmjlevx
  Resolved https://github.com/modAL-python/modAL.git to commit bba6f6fd00dbb862b1e09259b78caf6cffa2e755
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.8/125.8 kB 2.4 MB/s eta 0:00:00
  Created wheel for modAL-python: filename=modAL_python-0.4.2-py3-none-any.whl size=32650 sha256=cf16c3c42e3433774c07bcdb25c0afb46b9a93bb23f8be631c835f1ad0dd1557
  Stored in directory: /tmp/pip-ephem-wheel-cache-z2_cp36k/wheels/d9/fb/59/7deb61b460c1c36394cd093758986ff7d36f71352dcb2e02c5
Successfully built modAL-python
  Attempting uninstall: skorch
    Found existing installation: skorch 0.15.0
    Uninstalling skorch-0.15.0:
      Successfully uninstalled skorch-0.15.0


In [ ]:
import torch
from torch import nn
from skorch import NeuralNetClassifier

# build class for the skorch API
class Torch_Model(nn.Module):
    def __init__(self,):
        super(Torch_Model, self).__init__()
        self.convs = nn.Sequential(
                                nn.Conv2d(1,32,3),
                                nn.ReLU(),
                                nn.Conv2d(32,64,3),
                                nn.ReLU(),
                                nn.MaxPool2d(2),
                                nn.Dropout(0.25)
        )
        self.fcs = nn.Sequential(
                                nn.Linear(12*12*64,128),
                                nn.ReLU(),
                                nn.Dropout(0.5),
                                nn.Linear(128,10),
        )

    def forward(self, x):
        out = x
        out = self.convs(out)
        out = out.view(-1,12*12*64)
        out = self.fcs(out)
        return out

In [ ]:
# create the classifier
device = "cuda" if torch.cuda.is_available() else "cpu"
classifier = NeuralNetClassifier(Torch_Model,
                                 criterion=nn.CrossEntropyLoss,
                                 optimizer=torch.optim.Adam,
                                 train_split=None,
                                 verbose=1,
                                 device=device)

In [ ]:
import numpy as np
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from torchvision.datasets import MNIST


mnist_data = MNIST('.', download=True, transform=ToTensor())
dataloader = DataLoader(mnist_data, shuffle=True, batch_size=60000)
X, y = next(iter(dataloader))

# read training data
X_train, X_test, y_train, y_test = X[:50000], X[50000:], y[:50000], y[50000:]
X_train = X_train.reshape(50000, 1, 28, 28)
X_test = X_test.reshape(10000, 1, 28, 28)

# assemble initial data
n_initial = 1000
initial_idx = np.random.choice(range(len(X_train)), size=n_initial, replace=False)
X_initial = X_train[initial_idx]
y_initial = y_train[initial_idx]

# generate the pool
# remove the initial data from the training dataset
X_pool = np.delete(X_train, initial_idx, axis=0)[:5000]
y_pool = np.delete(y_train, initial_idx, axis=0)[:5000]

In [ ]:
from modAL.models import ActiveLearner

# initialize ActiveLearner
learner = ActiveLearner(
    estimator=classifier,
    X_training=X_initial, y_training=y_initial,
)

  epoch    train_loss     dur
-------  ------------  ------
      1        3.4760  2.4251
      2        2.3063  2.0536
      3        2.3036  4.6801
      4        2.3020  4.6826
      5        2.3003  3.6384
      6        2.2994  6.3140
      7        2.2981  2.1004
      8        2.2990  1.9690
      9        2.3004  1.9457
     10        2.2988  2.5454


In [ ]:
# the active learning loop
n_queries = 10
for idx in range(n_queries):
    print('Query no. %d' % (idx + 1))
    query_idx, query_instance = learner.query(X_pool, n_instances=100)
    learner.teach(
        X=X_pool[query_idx], y=y_pool[query_idx], only_new=True,
    )
    # remove queried instance from pool
    X_pool = np.delete(X_pool, query_idx, axis=0)
    y_pool = np.delete(y_pool, query_idx, axis=0)

Query no. 1
Re-initializing module.
Re-initializing criterion.
Re-initializing optimizer.
  epoch    train_loss     dur
-------  ------------  ------
      1        2.3075  0.2438
      2        5.0375  0.2403
      3        2.6199  0.2367
      4        2.2818  0.2382
      5        2.2579  0.2053
      6        2.1474  0.2012
      7        2.0206  0.1988
      8        1.7814  0.2072
      9        1.6168  0.2024
     10        1.3738  0.2157
Query no. 2
Re-initializing module.
Re-initializing criterion.
Re-initializing optimizer.
  epoch    train_loss     dur
-------  ------------  ------
      1        2.2824  0.2554
      2       20.2135  0.2354
      3        5.2120  0.2301
      4        2.2709  0.2212
      5        2.2671  0.2298
      6        2.2632  0.3001
      7        2.2582  0.3777
      8        2.2483  0.3771
      9        2.2390  0.3527
     10        2.2290  0.3407
Query no. 3
Re-initializing module.
Re-initializing criterion.
Re-initializing optimizer.
  epoch   